In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score, auc, confusion_matrix, precision_score, recall_score, roc_curve,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling

In [ ]:
train = pd.read_csv("../input/titanic/train.csv")
test =  pd.read_csv("../input/titanic/test.csv")

In [ ]:
data = [train, test]

In [ ]:
test.index

In [ ]:
train.info()
print('_'*40)
test.info()

In [ ]:
#pandas_profiling.profileReport(train)
print(train.shape)
print(test.shape)

In [ ]:
train.describe()

In [ ]:
#What is the distribution of categorical features?
train.describe(include=['O'])

In [ ]:
#count of survieved passenge=1 and not Survived=0
sns.countplot(x='Survived',data=train)

In [ ]:
#count of survieved passenge=1 and not Survived=0 w.r.t sex(m/f)
sns.countplot(x='Survived', hue = 'Sex',data=train)

In [ ]:
#which class passenger survised mostly
sns.barplot(y='Survived', x = 'Pclass',data=train)

In [ ]:
#visualize and find out the avg age by pclass
plt.figure(figsize=(10,10))
sns.boxplot(y='Age', x = 'Pclass',data=train, palette='winter')

In [ ]:
#function to impute the missing values in age column on the basis of class
def imput_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return (train[train['Pclass']==Pclass]['Age'].mean())
    else:
        return Age

In [ ]:
#for test data
def imput_age1(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        return (test[test['Pclass']==Pclass]['Age'].mean())
    else:
        return Age

In [ ]:
#drop cabin column becaue is catgrical data
train['Age'] = train[['Pclass','Age']].apply(imput_age,axis=1)
test['Age'] = test[['Pclass','Age']].apply(imput_age1,axis=1)

In [ ]:
#fill embark missing value with most frequent value
most_comon_value = 'S'

for i in train:
    train['Embarked']=train['Embarked'].fillna(most_comon_value)
    

In [ ]:
#convert categrical variable to demmy variable
Sex = pd.get_dummies(train['Sex'],drop_first=True)
Embarked = pd.get_dummies(train['Embarked'],drop_first=True)
Pclass = pd.get_dummies(train['Pclass'],drop_first=True)


In [ ]:
train.drop(['Sex','Pclass','Embarked','Name','Ticket','PassengerId'],axis=1, inplace=True)

In [ ]:
train.drop('Cabin',axis=1, inplace=True)
test.drop('Cabin',axis=1, inplace=True)


In [ ]:
Embarked.isnull().sum()

In [ ]:
Sex.shape

In [ ]:
train_f = pd.concat([train,Sex,Embarked,Pclass],axis=1)
train_f.head()

In [ ]:
#create feature
x_train=train_f.drop('Survived',axis=1)
y_train=train_f['Survived']

In [ ]:
x_train.shape, y_train.shape, 

In [ ]:
#fill na value in test data
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)
test.head()

In [ ]:
#REMOVE USUSAL CODE
test.drop(['Name','Ticket','PassengerId'],axis=1, inplace=True)

In [ ]:
test.head(2)

In [ ]:
#convert categrical variable to demmy variable
Sex = pd.get_dummies(test['Sex'],drop_first=True)
Embarked = pd.get_dummies(test['Embarked'],drop_first=True)
Pclass = pd.get_dummies(test['Pclass'],drop_first=True)


In [ ]:
test_f = pd.concat([test,Sex,Embarked,Pclass],axis=1)
test_f.head()

In [ ]:
#REMOVE USUSAL CODE
test_f.drop(['Pclass','Sex','Embarked'],axis=1, inplace=True)

In [ ]:
test_f.head(3)

In [ ]:
x_test=test_f

In [ ]:
x_test.shape

In [ ]:
#Building the Model 

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(x_train,y_train)

In [ ]:
model.score(x_train,y_train)   # Accuracy score for train

In [ ]:
y_pred_train =model.predict(x_train)

In [ ]:
accuracy_score(y_train, y_pred_train)    # calculates the accuracy score

In [ ]:
summary =classification_report(y_train, y_pred_train)
print(summary)

In [ ]:
#Predictions on Test Data 

In [ ]:
pred_test = model.predict(x_test)
#pred_test

In [ ]:
    titanic_submission = pd.DataFrame({"PassengerId": test.index, "Survived": pred_test })

titanic_submission.to_csv("titanic.csv", index=False)

In [ ]:
#DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from IPython.display import Image  
from sklearn.model_selection import GridSearchCV


In [ ]:
model_tree=DecisionTreeClassifier(max_depth=4)
model_tree.fit(x_train,y_train)

In [ ]:
export_graphviz(model_tree,out_file='Regtree.dot',feature_names=x_train.columns)
! dot -Tpng Regtree.dot -o Regtree.png
Image('Regtree.png')

In [ ]:
y_pred_train=model_tree.predict(x_train)

In [ ]:
accuracy_score(y_train,y_pred_train)

In [ ]:
y_pred_tree = model_tree.predict(x_test)

In [ ]:
print(y_pred_tree)

In [ ]:
data = list(y_pred_tree)
data 
np.average(data)

In [ ]:
titanic_submission = pd.DataFrame({"PassengerId": test.index, "Survived": y_pred_tree })

titanic_submission.to_csv("titanic_tree.csv", index=False)

In [ ]:
#333purning a tree
parameters = {'max_depth': [1,2,3,4,5,6,7,8,9,10]}
grid = GridSearchCV(model_tree,parameters,cv=10,return_train_score=False,scoring='accuracy')

In [ ]:
grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
#building new parameter base on params
model_prune = DecisionTreeClassifier(max_depth=9,random_state=42)
model_prune.fit(x_train,y_train)

In [ ]:
y_pred_test=model_prune.predict(x_test)

In [ ]:

data = list(y_pred_test)
data 
np.average(data)

In [ ]:
titanic_submission = pd.DataFrame({"PassengerId": test.index, "Survived": y_pred_test })

titanic_submission.to_csv("titanic1.csv", index=False)